### An integer programming approach to solve for the optimal discount strategy that maximizes the total revenue in the [Retail Game](http://www.randhawa.us/games/retailer/nyu.html).

### EDA

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_excel('Sales-Data.xlsx')

In [3]:
df

,Item,Week,Price,Sales,Remaining Inventory
0,1.0,1.0,60.0,57.0,1943.0
1,NaN,2.0,60.0,98.0,1845.0
2,NaN,3.0,60.0,55.0,1790.0
3,NaN,4.0,60.0,41.0,1749.0
4,NaN,5.0,60.0,60.0,1689.0
5,NaN,6.0,60.0,39.0,1650.0
6,NaN,7.0,54.0,106.0,1544.0
7,NaN,8.0,54.0,55.0,1489.0
8,NaN,9.0,54.0,64.0,1425.0
9,NaN,10.0,54.0,43.0,1382.0


### The given dataset contains 15 trials of the game, where for each run the player maintains full price for a couple of weeks and applies a discount until the end of the game. More specifically,
- Trial 1 - 5 (Group 1): full price from Week 1 to Week 6 and 10% discount from Week 7 to Week 15
- Trial 6 - 10 (Group 2): full price from Week 1 to Week 7 and 20% discount from Week 8 to Week 15
- Trial 11 - 15 (Group 3): full price from Week 1 to Week 9 and 40% discount from Week 10 to Week 15

### We can use these historical data to calculate estimated average sales for each discount to derive the demand lifts. Let's start with the 90% discount.

In [4]:
demand_lift_list = []
for i in range(5):
    start_idx, end_idx = i*17, i*17+15
#     print(df.iloc[start_idx:end_idx])
    avg_sales_groupby_price = df.iloc[start_idx:end_idx].groupby(by="Price").mean()
    print(avg_sales_groupby_price)
    demand_lift = avg_sales_groupby_price.iloc[0]['Sales']/avg_sales_groupby_price.iloc[1]['Sales']
    
    demand_lift_list.append(demand_lift)
    
    print("\n Demand lift from full price to 10% discount: " + str(demand_lift))
    print("\n")
    

       Item  Week      Sales  Remaining Inventory
Price                                            
54.0    NaN  11.0  76.000000          1257.666667
60.0    1.0   3.5  58.333333          1777.666667

 Demand lift from full price to 10% discount: 1.3028571428571427


       Item  Week       Sales  Remaining Inventory
Price                                             
54.0    NaN  11.0  144.000000           652.444444
60.0    2.0   3.5  107.666667          1608.000000

 Demand lift from full price to 10% discount: 1.3374613003095974


       Item  Week      Sales  Remaining Inventory
Price                                            
54.0    NaN  11.0  82.333333          1223.888889
60.0    3.0   3.5  59.333333          1763.833333

 Demand lift from full price to 10% discount: 1.3876404494382022


       Item  Week      Sales  Remaining Inventory
Price                                            
54.0    NaN  11.0  77.888889          1284.777778
60.0    4.0   3.5  61.166667          1801

In [5]:
round(np.mean(demand_lift_list),2)

1.31

In [6]:
round(np.std(demand_lift_list),2)

0.05

### We observe that although the average sales varies among different runs, but the demand lift (price elasticity) is pretty much constant ($\sigma = 0.05$). Now, we repeat this process for 20\% discount and 40% discount. <span style="color:red"> However, we do notice that there are cases where the T-shirts sold out before week 15, so to derive the real demands we use the data before the soldout week. </span>

In [7]:
demand_lift_list = []
for i in range(5,10):
    start_idx, end_idx = i*17, i*17+15

    trail = df.iloc[start_idx:end_idx]
    trail_without_soldout = trail[trail["Remaining Inventory"] != 0]
    
#     print(trail_without_soldout)
    avg_sales_groupby_price = trail_without_soldout.groupby(by="Price").mean()
    print(avg_sales_groupby_price)
    demand_lift = avg_sales_groupby_price.iloc[0]['Sales']/avg_sales_groupby_price.iloc[1]['Sales']
    
    demand_lift_list.append(demand_lift)
    
    print("\n Demand lift from full price to 20% discount: " + str(demand_lift))
    print("\n")
    

       Item  Week       Sales  Remaining Inventory
Price                                             
48.0    NaN  10.0  209.400000           552.400000
60.0    6.0   4.0  114.142857          1535.857143

 Demand lift from full price to 20% discount: 1.8345431789737172


       Item  Week       Sales  Remaining Inventory
Price                                             
48.0    NaN  11.5  119.625000          1022.000000
60.0    7.0   4.0   67.428571          1728.428571

 Demand lift from full price to 20% discount: 1.7740995762711864


       Item  Week  Sales  Remaining Inventory
Price                                        
48.0    NaN  11.5  96.75          1188.000000
60.0    8.0   4.0  53.00          1787.142857

 Demand lift from full price to 20% discount: 1.8254716981132075


       Item  Week       Sales  Remaining Inventory
Price                                             
48.0    NaN  11.5  131.875000           877.375000
60.0    9.0   4.0   73.714286          1697.428571


In [8]:
round(np.mean(demand_lift_list),2)

1.73

In [9]:
round(np.std(demand_lift_list),2)

0.15

In [10]:
demand_lift_list = []
for i in range(10,15):
    start_idx, end_idx = i*17, i*17+15

    trail = df.iloc[start_idx:end_idx]
    trail_without_soldout = trail[trail["Remaining Inventory"] != 0]
    
#     print(trail_without_soldout)
    avg_sales_groupby_price = trail_without_soldout.groupby(by="Price").mean()
    print(avg_sales_groupby_price)
    demand_lift = avg_sales_groupby_price.iloc[0]['Sales']/avg_sales_groupby_price.iloc[1]['Sales']
    
    demand_lift_list.append(demand_lift)
    
    print("\n Demand lift from full price to 40% discount: " + str(demand_lift))
    print("\n")

       Item  Week       Sales  Remaining Inventory
Price                                             
36.0    NaN  11.5  264.250000           372.500000
60.0   11.0   5.0  100.444444          1541.888889

 Demand lift from full price to 40% discount: 2.630807522123894


       Item  Week       Sales  Remaining Inventory
Price                                             
36.0    NaN  12.5  188.500000           676.833333
60.0   12.0   5.0   64.111111          1671.333333

 Demand lift from full price to 40% discount: 2.940207972270364


       Item  Week       Sales  Remaining Inventory
Price                                             
36.0    NaN  12.5  196.666667           773.166667
60.0   13.0   5.0   65.555556          1693.444444

 Demand lift from full price to 40% discount: 3.0


       Item  Week       Sales  Remaining Inventory
Price                                             
36.0    NaN  12.5  164.000000           898.666667
60.0   14.0   5.0   61.444444          1660.2222

In [11]:
round(np.mean(demand_lift_list),2)

2.81

In [12]:
round(np.std(demand_lift_list),2)

0.14

### Now, we have the demand lifts for the three discounts.
- 10\% discount: 1.31
- 20\% discount: 1.73
- 40\% discount: 2.81

## Optimization Model Build-up

### Info given and assumptions:
- T-shirts are sold at full price in week 1
- All items remaining at the end of the sales horizon will be discarded at 0 value.
- demand lift is given as $1.31, 1.73$ and $2.81$
- price can only be decreased

### Notations:
- $p$: full price
- $T$: time horizon (weeks)
- $D$: available discount options
- $s$: initial inventory
- $a$: sales at full price in Week 1 (given at the beginning of each game)

### Parameters:
- $p = 60$
- $T = \{1, \cdots, 15\}$
- $D = \{1, 0.9, 0.8, 0.6\}$
- $s = 2000$

### Decision variables:
- $x_d$: number of weeks that the T-shirts sell at discount $d$

####  Objective:
$$\sum_{d\in D} (a \cdot  p \cdot d \cdot x_d)$$

#### Constraints:
- T-shirts are sold at full price in week 1
$$x_1 \ge 1$$
- Total sales constraint
$$a(x_1 + 1.31x_{0.9} + 1.73x_{0.8} + 2.81x_{0.6}) \le 2000$$
- Time horizon constraint
$$x_1 + x_{0.9} + x_{0.8} + x_{0.6} \le 15$$
- Integrality
$$x_d \in \mathbb{N}$$

In [13]:
import pulp as plp
import numpy as np

In [14]:
# Define problem
model = plp.LpProblem("Retail_Game", plp.LpMaximize)

In [15]:
I = range(4)
X = plp.LpVariable.dicts(name='X', indexs=(I),lowBound=0, cat=plp.LpInteger)

In [16]:
discounted_price = [60,54,48,36]
demand_lift = [1,1.31,1.73,2.81]
a = 65

In [17]:
model += plp.lpSum(a*discounted_price[i]*demand_lift[i]*X[i] for i in I)

In [18]:
model += X[1] >= 1
model += plp.lpSum(a*demand_lift[i]*X[i] for i in I) <= 2000
model += plp.lpSum(X[i] for i in I) <= 15

In [19]:
model.solve()

1

In [20]:
# The status of the solution is printed to the screen
print("="*30,"\nSolution Status:", plp.LpStatus[model.status])

# Results
obj = plp.value(model.objective)
print("The result is: ${}".format(obj))

print("Optimal Soultion:")
for v in model.variables():
    print(v.name, "=", v.varValue)

Solution Status: Optimal
The result is: $85254.0
Optimal Soultion:
X_0 = 0.0
X_1 = 2.0
X_2 = 8.0
X_3 = 5.0


In [25]:
import pulp as plp
import numpy as np
def optimizer(a):
    model = plp.LpProblem("Retail_Game", plp.LpMaximize)
    I = range(4)
    X = plp.LpVariable.dicts(name='X', indexs=(I),lowBound=0, cat=plp.LpInteger)
    discounted_price = [60,54,48,36]
    demand_lift = [1,1.31,1.73,2.81]
    model += plp.lpSum(a*discounted_price[i]*demand_lift[i]*X[i] for i in I)
    model += X[1] >= 1
    model += plp.lpSum(a*demand_lift[i]*X[i] for i in I) <= 2000
    model += plp.lpSum(X[i] for i in I) <= 15
    model.solve()
    print("="*30,"\nSolution Status:", plp.LpStatus[model.status])

    # Results
    obj = plp.value(model.objective)
    print("The result is: ${}".format(obj))
    values=[]
    print("Optimal Soultion:")
    for v in model.variables():
        print(v.name, "=", v.varValue)
        values.append(v.varValue)
    return(model,values)
optimizer(100)

Solution Status: Optimal
The result is: $106578.0
Optimal Soultion:
X_0 = 3.0
X_1 = 9.0
X_2 = 3.0
X_3 = 0.0


(Retail_Game:
 MAXIMIZE
 6000*X_0 + 7074.0*X_1 + 8304.0*X_2 + 10116.0*X_3 + 0.0
 SUBJECT TO
 _C1: X_1 >= 1
 
 _C2: 100 X_0 + 131 X_1 + 173 X_2 + 281 X_3 <= 2000
 
 _C3: X_0 + X_1 + X_2 + X_3 <= 15
 
 VARIABLES
 0 <= X_0 Integer
 0 <= X_1 Integer
 0 <= X_2 Integer
 0 <= X_3 Integer,
 [3.0, 9.0, 3.0, 0.0])